# 04 - ETL: Establecimientos de Salud (RAS)

**Objetivo:** Procesar datos de establecimientos de salud y asignarlos a parroquias

**Estrategia:** JOIN por nombres (provincia-cantón-parroquia) - sin coordenadas geográficas

**Entradas:**
- CSV RAS: `RAS_2020.csv` (Registro de Atención en Salud - MSP)
- CSV procesado: `parroquias_con_etnia.csv` (del Notebook 03)

**Salidas:**
- CSV: `establecimientos_salud_procesados.csv` (establecimientos con código de parroquia)
- CSV: `parroquias_con_salud.csv` (tabla agregada por parroquia)

---

## 1. Setup e Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
import unicodedata
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Rutas
BASE_DIR = Path('../data')
RAW_SALUD = BASE_DIR / 'raw' / 'msp_ras_2020'
PROCESSED_DIR = BASE_DIR / 'processed'

print("✅ Librerías cargadas")
print(f"📁 Directorio salud: {RAW_SALUD}")
print(f"📁 Directorio processed: {PROCESSED_DIR}")

✅ Librerías cargadas
📁 Directorio salud: ../data/raw/msp_ras_2020
📁 Directorio processed: ../data/processed


---
## 2. Cargar Datos RAS

In [2]:
# Cargar datos RAS (con separador punto y coma)
print("📂 Cargando datos de establecimientos de salud...")

ras = pd.read_csv(
    RAW_SALUD / 'RAS_2020.csv',
    encoding='utf-8',
    sep=';',
    on_bad_lines='skip'
)

print(f"✅ Archivo cargado")
print(f"📊 Dimensiones: {ras.shape}")
print(f"\n📋 Columnas de ubicación:")
print(['secuencial', 'prov_ubi', 'cant_ubi', 'parr_ubi', 'clase', 'entidad', 'sector'])

📂 Cargando datos de establecimientos de salud...
✅ Archivo cargado
📊 Dimensiones: (4136, 1931)

📋 Columnas de ubicación:
['secuencial', 'prov_ubi', 'cant_ubi', 'parr_ubi', 'clase', 'entidad', 'sector']


In [3]:
# Exploración inicial
print("🔍 EXPLORACIÓN INICIAL\n")
print("📊 Primeras filas:")
print(ras[['secuencial', 'prov_ubi', 'cant_ubi', 'parr_ubi', 'clase', 'entidad', 'sector']].head(10))

print(f"\n📊 Valores únicos:")
print(f"   Provincias: {ras['prov_ubi'].nunique()}")
print(f"   Cantones: {ras['cant_ubi'].nunique()}")
print(f"   Parroquias: {ras['parr_ubi'].nunique()}")
print(f"   Tipos de establecimiento: {ras['clase'].nunique()}")

🔍 EXPLORACIÓN INICIAL

📊 Primeras filas:
   secuencial   prov_ubi         cant_ubi  \
0           1    Bolívar        Chillanes   
1           2     Manabí      San Vicente   
2           3   Imbabura         Cotacahi   
3           4      Cañar          Biblian   
4           5       Loja           Macará   
5           6  Pichincha            Quito   
6           7   Cotopaxi           Pujilí   
7           8  Sucumbíos  Gonzalo Pizarro   
8           9     Manabí            Junín   
9          10       Loja         Quilanga   

                                    parr_ubi  \
0                                  Chillanes   
1                                      Canoa   
2    Vacas Galindo (Cab. En San Miguel Alto)   
3                                    Biblián   
4  Macará (Manuel Enrique Rengel Suquilanda)   
5                                   Puellaro   
6                                     Pujilí   
7                            Gonzalo Pizarro   
8                              

---
## 3. Limpiar y Filtrar Datos

In [4]:
# Seleccionar columnas relevantes
print("🔄 SELECCIONANDO COLUMNAS RELEVANTES\n")

ras_clean = ras[['secuencial', 'prov_ubi', 'cant_ubi', 'parr_ubi', 'clase', 'entidad', 'sector']].copy()

# Renombrar para claridad
ras_clean.columns = ['id_establecimiento', 'provincia', 'canton', 'parroquia', 'tipo_establecimiento', 'institucion', 'sector']

print("✅ Columnas seleccionadas y renombradas")
print(f"\n📊 Datos limpios: {ras_clean.shape}")

🔄 SELECCIONANDO COLUMNAS RELEVANTES

✅ Columnas seleccionadas y renombradas

📊 Datos limpios: (4136, 7)


In [5]:
# Filtrar registros con ubicación válida
print("🔄 FILTRANDO REGISTROS VÁLIDOS\n")

# Eliminar filas con ubicación nula
ras_validos = ras_clean[
    ras_clean['provincia'].notna() &
    ras_clean['canton'].notna() &
    ras_clean['parroquia'].notna()
].copy()

print(f"Filtro - Con ubicación válida: {len(ras_clean)} → {len(ras_validos)}")
print(f"\n✅ Total establecimientos válidos: {len(ras_validos)}")

🔄 FILTRANDO REGISTROS VÁLIDOS

Filtro - Con ubicación válida: 4136 → 4136

✅ Total establecimientos válidos: 4136


In [6]:
# Estandarizar nombres (mayúsculas, espacios)
print("🔄 ESTANDARIZANDO NOMBRES\n")

def limpiar_nombre(texto):
    if pd.isna(texto):
        return texto
    return str(texto).strip().upper()

ras_validos['provincia'] = ras_validos['provincia'].apply(limpiar_nombre)
ras_validos['canton'] = ras_validos['canton'].apply(limpiar_nombre)
ras_validos['parroquia'] = ras_validos['parroquia'].apply(limpiar_nombre)

print("✅ Nombres estandarizados")
print(f"\n📊 Muestra:")
print(ras_validos[['provincia', 'canton', 'parroquia']].head(10))

🔄 ESTANDARIZANDO NOMBRES

✅ Nombres estandarizados

📊 Muestra:
   provincia           canton                                  parroquia
0    BOLÍVAR        CHILLANES                                  CHILLANES
1     MANABÍ      SAN VICENTE                                      CANOA
2   IMBABURA         COTACAHI    VACAS GALINDO (CAB. EN SAN MIGUEL ALTO)
3      CAÑAR          BIBLIAN                                    BIBLIÁN
4       LOJA           MACARÁ  MACARÁ (MANUEL ENRIQUE RENGEL SUQUILANDA)
5  PICHINCHA            QUITO                                   PUELLARO
6   COTOPAXI           PUJILÍ                                     PUJILÍ
7  SUCUMBÍOS  GONZALO PIZARRO                            GONZALO PIZARRO
8     MANABÍ            JUNÍN                                      JUNÍN
9       LOJA         QUILANGA                                   QUILANGA


---
## 4. Diccionario de Mapeo y Normalización

Aplicamos la misma estrategia del Notebook 03

In [7]:
# Diccionario de mapeo (casos especiales)
print("🔧 CREANDO DICCIONARIO DE MAPEO\n")

MAPEO_PROVINCIAS = {
    'SANTO DOMINGO DE LOS TSÁCHILAS': 'STO DGO TSACHILAS',
    'SANTO DOMINGO DE LOS TSACHILAS': 'STO DGO TSACHILAS',
}

def aplicar_mapeo_provincia(nombre):
    if pd.isna(nombre):
        return nombre
    nombre_upper = str(nombre).upper().strip()
    return MAPEO_PROVINCIAS.get(nombre_upper, nombre_upper)

# Aplicar mapeo
ras_validos['provincia'] = ras_validos['provincia'].apply(aplicar_mapeo_provincia)

print("✅ Mapeo aplicado")

# Verificar Santo Domingo
santo_ras = ras_validos[ras_validos['provincia'].str.contains('STO DGO', na=False)]
print(f"\n🔍 Santo Domingo en RAS: {len(santo_ras)} establecimientos")

# 2️⃣ DESPUÉS: Mapeo de CANTONES
MAPEO_CANTONES = {
    'COTACAHI': 'COTACACHI',
}

def aplicar_mapeo_canton(nombre):
    if pd.isna(nombre):
        return nombre
    nombre_upper = str(nombre).upper().strip()
    return MAPEO_CANTONES.get(nombre_upper, nombre_upper)

# Aplicar mapeo de cantones
ras_validos['canton'] = ras_validos['canton'].apply(aplicar_mapeo_canton)

print("✅ Mapeo de cantones aplicado")

# Verificar
cotacachi_ras = ras_validos[ras_validos['canton'].str.contains('COTACACHI', na=False)]
print(f"🔍 COTACACHI en RAS: {len(cotacachi_ras)} establecimientos")



🔧 CREANDO DICCIONARIO DE MAPEO

✅ Mapeo aplicado

🔍 Santo Domingo en RAS: 112 establecimientos
✅ Mapeo de cantones aplicado
🔍 COTACACHI en RAS: 23 establecimientos


In [8]:
# Función de normalización (igual que Notebook 03)
def normalizar_nombre(nombre):
    """
    Normalización híbrida:
    - Quita tildes
    - Elimina caracteres especiales
    - Toma primera palabra significativa (>2 letras)
    """
    if pd.isna(nombre):
        return nombre
    
    nombre = str(nombre)
    
    # Quitar tildes/acentos
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    
    # Reemplazar separadores por espacio
    nombre = nombre.replace('/', ' ').replace('-', ' ')
    
    # Limpiar puntuación
    nombre = nombre.replace('.', '').replace(',', '')
    
    # Normalizar espacios
    nombre = ' '.join(nombre.split())
    nombre = nombre.upper().strip()
    
    # Tomar primera palabra significativa (más de 2 letras)
    palabras = nombre.split()
    for palabra in palabras:
        if len(palabra) > 2:
            return palabra
    
    return palabras[0] if palabras else nombre

print("✅ Función de normalización definida")

✅ Función de normalización definida


In [9]:
# Aplicar normalización
print("🔄 APLICANDO NORMALIZACIÓN\n")

ras_validos['provincia_norm'] = ras_validos['provincia'].apply(normalizar_nombre)
ras_validos['canton_norm'] = ras_validos['canton'].apply(normalizar_nombre)
ras_validos['parroquia_norm'] = ras_validos['parroquia'].apply(normalizar_nombre)

print("✅ Normalización aplicada")
print(f"\n📊 Ejemplos:")
print(ras_validos[['provincia', 'provincia_norm', 'parroquia', 'parroquia_norm']].head(10))

🔄 APLICANDO NORMALIZACIÓN

✅ Normalización aplicada

📊 Ejemplos:
   provincia provincia_norm                                  parroquia  \
0    BOLÍVAR        BOLIVAR                                  CHILLANES   
1     MANABÍ         MANABI                                      CANOA   
2   IMBABURA       IMBABURA    VACAS GALINDO (CAB. EN SAN MIGUEL ALTO)   
3      CAÑAR          CANAR                                    BIBLIÁN   
4       LOJA           LOJA  MACARÁ (MANUEL ENRIQUE RENGEL SUQUILANDA)   
5  PICHINCHA      PICHINCHA                                   PUELLARO   
6   COTOPAXI       COTOPAXI                                     PUJILÍ   
7  SUCUMBÍOS      SUCUMBIOS                            GONZALO PIZARRO   
8     MANABÍ         MANABI                                      JUNÍN   
9       LOJA           LOJA                                   QUILANGA   

  parroquia_norm  
0      CHILLANES  
1          CANOA  
2          VACAS  
3        BIBLIAN  
4         MACARA  
5     

---
## 5. Cargar Tabla Base de Parroquias

In [10]:
# Cargar tabla con etnia (del Notebook 03)
print("📂 Cargando tabla base de parroquias (con etnia)...")

parroquias_etnia = pd.read_csv(PROCESSED_DIR / 'parroquias_con_etnia.csv')

print(f"✅ Tabla cargada: {len(parroquias_etnia)} parroquias")
print(f"\n📋 Columnas:")
print(parroquias_etnia.columns.tolist())

📂 Cargando tabla base de parroquias (con etnia)...
✅ Tabla cargada: 1236 parroquias

📋 Columnas:
['codigo_dpa', 'nombre_provincia', 'nombre_canton', 'nombre_parroquia', 'centroide_lon', 'centroide_lat', 'area_km2', 'poblacion_total', 'poblacion_afro', 'pct_poblacion_afro']


In [11]:
# Aplicar MISMO mapeo y normalización a tabla base
print("🔄 NORMALIZANDO TABLA BASE (para consistency)\n")

# Mapeo
parroquias_etnia['nombre_provincia'] = parroquias_etnia['nombre_provincia'].apply(aplicar_mapeo_provincia)

# Normalización
parroquias_etnia['provincia_norm'] = parroquias_etnia['nombre_provincia'].apply(normalizar_nombre)
parroquias_etnia['canton_norm'] = parroquias_etnia['nombre_canton'].apply(normalizar_nombre)
parroquias_etnia['parroquia_norm'] = parroquias_etnia['nombre_parroquia'].apply(normalizar_nombre)

print("✅ Normalización aplicada a tabla base")

🔄 NORMALIZANDO TABLA BASE (para consistency)

✅ Normalización aplicada a tabla base


---
## 6. JOIN: RAS ↔ Parroquias

In [12]:
# Deduplicar RAS (por si hay duplicados)
print("🔍 VERIFICANDO DUPLICADOS EN RAS\n")

dup_antes = ras_validos.duplicated(subset=['provincia_norm', 'canton_norm', 'parroquia_norm']).sum()
print(f"Duplicados en RAS: {dup_antes}")

# No deduplicamos porque queremos CONTAR todos los establecimientos
print("\n💡 NO deduplicamos RAS - queremos contar todos los establecimientos")

🔍 VERIFICANDO DUPLICADOS EN RAS

Duplicados en RAS: 3054

💡 NO deduplicamos RAS - queremos contar todos los establecimientos


In [14]:
# JOIN
print("🔗 EJECUTANDO JOIN\n")
print("="*70)

# Merge con parroquias
ras_con_codigo = ras_validos.merge(
    parroquias_etnia[['codigo_dpa', 'provincia_norm', 'canton_norm', 'parroquia_norm']],
    on=['provincia_norm', 'canton_norm', 'parroquia_norm'],
    how='left'
)

# Contar asignaciones
asignados = ras_con_codigo['codigo_dpa'].notna().sum()
no_asignados = ras_con_codigo['codigo_dpa'].isna().sum()

print(f"📊 RESULTADOS DEL JOIN:")
print(f"   ✅ Establecimientos asignados: {asignados} ({asignados/len(ras_con_codigo)*100:.1f}%)")
print(f"   ⚠️  Sin asignar: {no_asignados} ({no_asignados/len(ras_con_codigo)*100:.1f}%)")
print(f"   📋 Total establecimientos: {len(ras_con_codigo)}")

print("\n" + "="*70)

🔗 EJECUTANDO JOIN

📊 RESULTADOS DEL JOIN:
   ✅ Establecimientos asignados: 4240 (91.2%)
   ⚠️  Sin asignar: 408 (8.8%)
   📋 Total establecimientos: 4648



In [15]:
# Explorar establecimientos sin asignar
if no_asignados > 0:
    print("🔍 ESTABLECIMIENTOS SIN ASIGNAR (primeros 10):\n")
    sin_asignar = ras_con_codigo[ras_con_codigo['codigo_dpa'].isna()]
    print(sin_asignar[['provincia', 'canton', 'parroquia', 'tipo_establecimiento']].head(10))
    
    print(f"\n📍 Distribución por provincia:")
    print(sin_asignar['provincia'].value_counts().head(10))

🔍 ESTABLECIMIENTOS SIN ASIGNAR (primeros 10):

            provincia                   canton  \
10           IMBABURA                  OTAVALO   
23             GUAYAS                  SALITRE   
38           LOS RÍOS                 BUENA FÉ   
39             GUAYAS  SAN JACINTO DE YAGUACHI   
40           IMBABURA    SAN MIGUEL DE URCUQUI   
58         ESMERALDAS               ESMERALDAS   
74           IMBABURA    SAN MIGUEL DE URCUQUI   
86   ZAMORA CHINCHIPE                 YANTZAZA   
88               LOJA                GONZANAMÁ   
116          COTOPAXI                LATACUNGA   

                                   parroquia              tipo_establecimiento  
10                                   OTAVALO  Dispensario Médico (Policlínico)  
23             GENERAL VERNAZA (DOS ESTEROS)  Dispensario Médico (Policlínico)  
38                   SAN JACINTO DE BUENA FÉ  Dispensario Médico (Policlínico)  
39          GRAL. PEDRO J. MONTERO (BOLICHE)  Dispensario Médico (Policlínico)

---
## 7. Agregar por Parroquia

In [16]:
# Contar establecimientos por parroquia
print("📊 AGREGANDO ESTABLECIMIENTOS POR PARROQUIA\n")

# Solo establecimientos asignados
ras_asignados = ras_con_codigo[ras_con_codigo['codigo_dpa'].notna()].copy()

# Contar por parroquia
establecimientos_por_parroquia = ras_asignados.groupby('codigo_dpa').size().reset_index(name='num_establecimientos')

print(f"✅ Agregación completada")
print(f"📊 Parroquias con al menos 1 establecimiento: {len(establecimientos_por_parroquia)}")
print(f"\n📊 Estadísticas:")
print(establecimientos_por_parroquia['num_establecimientos'].describe())

print(f"\n🔝 TOP 10 PARROQUIAS CON MÁS ESTABLECIMIENTOS:")
top_salud = establecimientos_por_parroquia.nlargest(10, 'num_establecimientos')
print(top_salud)

📊 AGREGANDO ESTABLECIMIENTOS POR PARROQUIA

✅ Agregación completada
📊 Parroquias con al menos 1 establecimiento: 1014

📊 Estadísticas:
count    1014.000000
mean        4.181460
std         8.266335
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max       214.000000
Name: num_establecimientos, dtype: float64

🔝 TOP 10 PARROQUIAS CON MÁS ESTABLECIMIENTOS:
      codigo_dpa  num_establecimientos
868       6150.0                   214
885       6265.0                    72
785       5625.0                    41
492       3325.0                    32
520       3475.0                    32
1003      7240.0                    32
1004      7245.0                    32
1005      7250.0                    32
740       5355.0                    28
989       7170.0                    25


---
## 8. Integrar con Tabla Base

In [17]:
# Merge con tabla base
print("🔗 INTEGRANDO DATOS DE SALUD CON TABLA BASE\n")

parroquias_con_salud = parroquias_etnia.merge(
    establecimientos_por_parroquia,
    on='codigo_dpa',
    how='left'
)

# Llenar NaN con 0 (parroquias sin establecimientos)
parroquias_con_salud['num_establecimientos'].fillna(0, inplace=True)
parroquias_con_salud['num_establecimientos'] = parroquias_con_salud['num_establecimientos'].astype(int)

# Eliminar columnas de normalización (no son necesarias en output final)
cols_drop = ['provincia_norm', 'canton_norm', 'parroquia_norm']
parroquias_con_salud.drop(cols_drop, axis=1, inplace=True, errors='ignore')

print(f"✅ Integración completada")
print(f"\n📊 RESUMEN:")
print(f"   Total parroquias: {len(parroquias_con_salud)}")
print(f"   Con establecimientos: {(parroquias_con_salud['num_establecimientos'] > 0).sum()}")
print(f"   Sin establecimientos: {(parroquias_con_salud['num_establecimientos'] == 0).sum()}")

🔗 INTEGRANDO DATOS DE SALUD CON TABLA BASE

✅ Integración completada

📊 RESUMEN:
   Total parroquias: 1236
   Con establecimientos: 1014
   Sin establecimientos: 222


---
## 9. Calcular Densidad de Establecimientos

In [18]:
# Calcular densidades
print("📊 CALCULANDO DENSIDAD DE ESTABLECIMIENTOS\n")

# Densidad por km²
parroquias_con_salud['densidad_establecimientos_km2'] = (
    parroquias_con_salud['num_establecimientos'] / parroquias_con_salud['area_km2']
).round(4)

# Densidad por población (establecimientos por cada 10,000 habitantes)
parroquias_con_salud['establecimientos_por_10k_hab'] = np.where(
    parroquias_con_salud['poblacion_total'].notna() & (parroquias_con_salud['poblacion_total'] > 0),
    (parroquias_con_salud['num_establecimientos'] / parroquias_con_salud['poblacion_total'] * 10000).round(2),
    np.nan
)

print("✅ Densidades calculadas")
print(f"\n📊 Estadísticas de densidad por población:")
print(parroquias_con_salud['establecimientos_por_10k_hab'].describe())

print(f"\n🔝 TOP 10 PARROQUIAS CON MAYOR DENSIDAD (por población):")
top_densidad = parroquias_con_salud.nlargest(10, 'establecimientos_por_10k_hab')[
    ['nombre_provincia', 'nombre_canton', 'nombre_parroquia', 
     'num_establecimientos', 'poblacion_total', 'establecimientos_por_10k_hab']
]
print(top_densidad.to_string())

📊 CALCULANDO DENSIDAD DE ESTABLECIMIENTOS

✅ Densidades calculadas

📊 Estadísticas de densidad por población:
count    847.000000
mean       7.579374
std        8.454732
min        0.000000
25%        2.350000
50%        4.930000
75%        9.500000
max       68.970000
Name: establecimientos_por_10k_hab, dtype: float64

🔝 TOP 10 PARROQUIAS CON MAYOR DENSIDAD (por población):
      nombre_provincia  nombre_canton        nombre_parroquia  num_establecimientos  poblacion_total  establecimientos_por_10k_hab
900          GALAPAGOS  SAN CRISTOBAL        ISLA SANTA MARIA                     1            145.0                         68.97
866   ZAMORA CHINCHIPE      CHINCHIPE               EL CHORRO                     1            152.0                         65.79
391    MORONA SANTIAGO     GUALAQUIZA                AMAZONAS                     1            165.0                         60.61
395    MORONA SANTIAGO     GUALAQUIZA                 ROSARIO                     3            570

---
## 10. Validaciones

In [19]:
print("✅ VALIDACIONES FINALES\n")

# 1. Total de registros
assert len(parroquias_con_salud) == 1236, "Debe haber 1236 parroquias"
print("✅ 1. Total registros correcto (1236)")

# 2. No hay valores negativos
assert (parroquias_con_salud['num_establecimientos'] >= 0).all(), "No debe haber valores negativos"
print("✅ 2. No hay valores negativos")

# 3. Suma de establecimientos
suma_parroquias = parroquias_con_salud['num_establecimientos'].sum()
print(f"✅ 3. Total establecimientos asignados: {suma_parroquias}")

# 4. Densidades válidas
con_poblacion = parroquias_con_salud['poblacion_total'].notna() & (parroquias_con_salud['poblacion_total'] > 0)
sin_densidad = parroquias_con_salud[con_poblacion & parroquias_con_salud['establecimientos_por_10k_hab'].isna()]
assert len(sin_densidad) == 0, "Parroquias con población deben tener densidad calculada"
print("✅ 4. Densidades calculadas correctamente")

print("\n🎉 Todas las validaciones pasaron exitosamente")

✅ VALIDACIONES FINALES

✅ 1. Total registros correcto (1236)
✅ 2. No hay valores negativos
✅ 3. Total establecimientos asignados: 4240
✅ 4. Densidades calculadas correctamente

🎉 Todas las validaciones pasaron exitosamente


---
## 11. Exportar Datos

In [20]:
# 1. Establecimientos con código de parroquia
output_establecimientos = PROCESSED_DIR / 'establecimientos_salud_procesados.csv'
ras_asignados.to_csv(output_establecimientos, index=False, encoding='utf-8')
print(f"✅ 1. Establecimientos procesados: {output_establecimientos.name}")

# 2. Tabla agregada por parroquia
output_parroquias = PROCESSED_DIR / 'parroquias_con_salud.csv'
parroquias_con_salud.to_csv(output_parroquias, index=False, encoding='utf-8')
print(f"✅ 2. Tabla de parroquias con salud: {output_parroquias.name}")

print("\n🎉 EXPORTACIÓN COMPLETADA")

✅ 1. Establecimientos procesados: establecimientos_salud_procesados.csv
✅ 2. Tabla de parroquias con salud: parroquias_con_salud.csv

🎉 EXPORTACIÓN COMPLETADA


---
## 12. Resumen Final

In [21]:
print("\n" + "="*70)
print("📊 RESUMEN DEL ETL - ESTABLECIMIENTOS DE SALUD")
print("="*70)

print("\n✅ TAREAS COMPLETADAS:")
print("  1. Datos RAS cargados y filtrados")
print("  2. Mapeo y normalización aplicados (incluyendo Santo Domingo)")
print("  3. JOIN por nombres exitoso")
print("  4. Agregación por parroquia")
print("  5. Densidades calculadas")
print("  6. Validaciones exitosas")
print("  7. Datos exportados")

print("\n📊 ESTADÍSTICAS FINALES:")
print(f"  • Total establecimientos RAS: {len(ras_validos)}")
print(f"  • Establecimientos asignados: {asignados} ({asignados/len(ras_validos)*100:.1f}%)")
print(f"  • Parroquias con establecimientos: {(parroquias_con_salud['num_establecimientos'] > 0).sum()}")
print(f"  • Parroquias sin establecimientos: {(parroquias_con_salud['num_establecimientos'] == 0).sum()}")
print(f"  • Promedio establecimientos/parroquia: {parroquias_con_salud['num_establecimientos'].mean():.2f}")

print("\n📁 ARCHIVOS GENERADOS:")
print(f"  • data/processed/establecimientos_salud_procesados.csv")
print(f"  • data/processed/parroquias_con_salud.csv")

print("\n🚀 PRÓXIMO PASO:")
print("  Notebook 05: ETL Infraestructura Petrolera")

print("\n" + "="*70)


📊 RESUMEN DEL ETL - ESTABLECIMIENTOS DE SALUD

✅ TAREAS COMPLETADAS:
  1. Datos RAS cargados y filtrados
  2. Mapeo y normalización aplicados (incluyendo Santo Domingo)
  3. JOIN por nombres exitoso
  4. Agregación por parroquia
  5. Densidades calculadas
  6. Validaciones exitosas
  7. Datos exportados

📊 ESTADÍSTICAS FINALES:
  • Total establecimientos RAS: 4136
  • Establecimientos asignados: 4240 (102.5%)
  • Parroquias con establecimientos: 1014
  • Parroquias sin establecimientos: 222
  • Promedio establecimientos/parroquia: 3.43

📁 ARCHIVOS GENERADOS:
  • data/processed/establecimientos_salud_procesados.csv
  • data/processed/parroquias_con_salud.csv

🚀 PRÓXIMO PASO:
  Notebook 05: ETL Infraestructura Petrolera

